In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib

In [2]:
file_path = "c:\\Users\\josen\\Documents\\MASTER\\TFM\\SmartLifter.csv"
data = pd.read_csv(file_path)

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\josen\\Documents\\MASTER\\TFM\\SmartLifter.csv'

In [129]:
print("Primeras filas del dataset:")
print(data.head())

Primeras filas del dataset:
   user_id  Type_x       Level_x  exercise_id  Age  Gender  Weight_kg  \
0        2  Cardio  Intermediate          149   32  Female       68.1   
1        2  Cardio  Intermediate          349   32  Female       68.1   
2        2  Cardio  Intermediate          884   32  Female       68.1   
3        2  Cardio  Intermediate          993   32  Female       68.1   
4        2  Cardio  Intermediate         1729   32  Female       68.1   

   Height_m  Workout_time  Calories_Burned  ... Fat_Percentage  Frequency  \
0      1.66          1.11            677.0  ...           33.4          4   
1      1.66          1.11            677.0  ...           33.4          4   
2      1.66          1.11            677.0  ...           33.4          4   
3      1.66          1.11            677.0  ...           33.4          4   
4      1.66          1.11            677.0  ...           33.4          4   

        Level_y    BMI                  Title    Type     BodyPart  \


In [130]:
# 2. Preprocesar los datos
# Seleccionar las columnas relevantes
columns_to_use = ['Level_x', 'Age', 'Weight_kg', 'Height_m', 'Type', 'BodyPart', 'Equipment', 'Level', 'BodyGroup']
data = data[columns_to_use + ['Title']]

# Codificar variables categóricas
label_encoders = {}
for col in ['Level_x', 'Type', 'BodyPart', 'Equipment', 'Level', 'BodyGroup', "Age"]:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Normalizar las variables continuas
scaler = StandardScaler()
data[['Age', 'Weight_kg', 'Height_m']] = scaler.fit_transform(data[['Age', 'Weight_kg', 'Height_m']])

# Mostrar los datos preprocesados
print("\nDatos preprocesados:")
print(data.head())


Datos preprocesados:
   Level_x       Age  Weight_kg  Height_m  Type  BodyPart  Equipment  Level  \
0        1 -0.623482  -0.214869 -0.499801     0         0         11      1   
1        1 -0.623482  -0.214869 -0.499801     0         0          2      1   
2        1 -0.623482  -0.214869 -0.499801     0         4         11      1   
3        1 -0.623482  -0.214869 -0.499801     0         5          2      1   
4        1 -0.623482  -0.214869 -0.499801     0        11          9      1   

   BodyGroup                  Title  
0          1            Roc Everest  
1          1  HM Running Man Crunch  
2          2         Agility Ladder  
3          4               Swimming  
4          0                  Rower  


In [131]:
# 3. Dividir los datos en entrenamiento y prueba
X = data[['Level_x', 'Age', 'Weight_kg', 'Height_m', 'Type', 'BodyPart', 'Equipment', 'Level', 'BodyGroup']]
y = data['Title']  # El título del ejercicio es la etiqueta

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [132]:
# 4. Optimizar el uso de memoria y entrenar el modelo Random Forest

# Reducir el tamaño del dataset para entrenamiento y prueba
X_train_sample = X_train.sample(frac=0.05, random_state=42)  # Usar solo el 5% de los datos
y_train_sample = y_train.loc[X_train_sample.index]

X_test_sample = X_test.sample(frac=0.05, random_state=42)  # Usar solo el 5% de los datos
y_test_sample = y_test.loc[X_test_sample.index]

# Reducir el número de estimadores y profundidad máxima para disminuir el uso de memoria
model = RandomForestClassifier(n_estimators=20, max_depth=10, random_state=42)
model.fit(X_train_sample, y_train_sample)

# Evaluar el modelo
y_pred_sample = model.predict(X_test_sample)
accuracy_sample = accuracy_score(y_test_sample, y_pred_sample)
print(f"\nPrecisión del modelo (muestra): {accuracy_sample:.2f}")


Precisión del modelo (muestra): 0.05


In [133]:
# 5. Guardar el modelo y los preprocesadores
joblib.dump(model, 'random_forest_model.pkl')
joblib.dump(label_encoders, 'label_encoders.pkl')
joblib.dump(scaler, 'scaler.pkl')

print("\nModelo y preprocesadores guardados exitosamente.")


Modelo y preprocesadores guardados exitosamente.


In [134]:
# 6. Función para recomendar ejercicios
def recomendar_ejercicios(model, user_features, label_encoders, scaler, num_ejercicios=5, dias=7):
    """
    Recomienda ejercicios basados en las características del usuario.

    Args:
        model: Modelo Random Forest entrenado.
        user_features: Diccionario con las características del usuario (nivel, edad, peso, altura).
        label_encoders: Diccionario con los codificadores de las variables categóricas.
        scaler: Escalador para normalizar las características continuas.
        num_ejercicios: Número de ejercicios por día.
        dias: Número de días en la rutina.

    Returns:
        Una rutina semanal de ejercicios.
    """
    user_data = pd.DataFrame([user_features])
    
    # Codificar las columnas categóricas
    for col in ['Level_x', 'Age', 'Weight_kg', 'Height_m', 'Type', 'Frequency']:
        user_data[col] = label_encoders[col].transform(user_data[col])
    
    # Normalizar las columnas continuas
    user_data[['Level_x', 'Age', 'Weight_kg', 'Height_m', 'Type', 'Frequency']] = scaler.transform(user_data[['Level_x', 'Age', 'Weight_kg', 'Height_m', 'Type', 'Frequency']])
    
    # Predecir los ejercicios
    predicciones = model.predict_proba(user_data)
    top_indices = np.argsort(predicciones, axis=1)[:, -num_ejercicios*dias:][0]
    
    # Obtener los títulos de los ejercicios recomendados
    ejercicios_recomendados = y.iloc[top_indices].values
    
    # Asegurar que no se repitan ejercicios el mismo día ni al día siguiente
    rutina_semanal = []
    usados = set()
    for i in range(dias):
        dia_ejercicios = []
        for ejercicio in ejercicios_recomendados:
            if ejercicio not in usados and len(dia_ejercicios) < num_ejercicios:
                dia_ejercicios.append(ejercicio)
                usados.add(ejercicio)
        rutina_semanal.append(dia_ejercicios)
    
    return rutina_semanal

In [147]:
# 7. Ejemplo de uso
user_features = {
    'Level_x': 'Beginner',
    'Age': 23,
    'Weight_kg': 70,
    'Height_m': 1.70,
    'Type': 'Strength',
    'Frequency': 3,  
}

# Cargar el modelo y los preprocesadores
loaded_model = joblib.load('random_forest_model.pkl')
loaded_label_encoders = joblib.load('label_encoders.pkl')
loaded_scaler = joblib.load('scaler.pkl')


# Mostrar la rutina semanal
print("\nRutina semanal recomendada:")

# Use the loaded model, label encoders, and scaler to recommend exercises
rutina_semanal = recomendar_ejercicios(loaded_model, user_features, loaded_label_encoders, loaded_scaler)

for dia, ejercicios in enumerate(rutina, 1):
    print(f"Día {dia}: {', '.join(ejercicios)}")


Rutina semanal recomendada:


KeyError: 'Weight_kg'

In [ ]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib

# 1. Cargar el dataset
file_path = "c:\\Users\\josen\\Documents\\MASTER\\TFM\\SmartLifter.csv"
data = pd.read_csv(file_path)

# 2. Preprocesar los datos
# Seleccionar las columnas relevantes
columns_to_use = ['Level_x', 'Age', 'Weight_kg', 'Height_m', 'Type', 'Frequency']
data = data[columns_to_use + ['Title']]

# Filtrar ejercicios según el nivel y tipo
def filtrar_ejercicios(data, level, exercise_type):
    return data[(data['Level_x'] == level) & (data['Type'] == exercise_type)]

# Codificar variables categóricas
label_encoders = {}
for col in ['Level_x', 'Type']:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Normalizar las variables continuas
scaler = StandardScaler()
data[['Age', 'Weight_kg', 'Height_m', 'Frequency', "Type", 'Level_x']] = scaler.fit_transform(data[['Age', 'Weight_kg', 'Height_m', 'Frequency', "Type", 'Level_x']])

# 3. Dividir los datos en entrenamiento y prueba
X = data[['Age', 'Weight_kg', 'Height_m', 'Frequency', "Type", 'Level_x']]
y = data['Title']  # El título del ejercicio es la etiqueta

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Entrenar el modelo Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluar el modelo
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nPrecisión del modelo: {accuracy:.2f}")

# 5. Guardar el modelo y los preprocesadores
joblib.dump(model, 'random_forest_model.pkl')
joblib.dump(label_encoders, 'label_encoders.pkl')
joblib.dump(scaler, 'scaler.pkl')


# 6. Función para recomendar ejercicios
def recomendar_ejercicios(model, user_features, label_encoders, scaler, num_ejercicios=6, dias=7):
    """
    Recomienda ejercicios basados en las características del usuario.

    Args:
        model: Modelo Random Forest entrenado.
        user_features: Diccionario con las características del usuario (nivel, edad, peso, altura, frecuencia).
        label_encoders: Diccionario con los codificadores de las variables categóricas.
        scaler: Escalador para normalizar las características continuas.
        num_ejercicios: Número de ejercicios por día.
        dias: Número de días en la rutina.

    Returns:
        Una rutina semanal de ejercicios.
    """
    user_data = pd.DataFrame([user_features])
    
    # Codificar las columnas categóricas
    for col in ['Level_x', 'Type']:
        user_data[col] = label_encoders[col].transform(user_data[col])
    
    # Normalizar las columnas continuas
    user_data[['Age', 'Weight_kg', 'Height_m', 'Frequency', "Type", 'Level_x']] = scaler.transform(user_data[['Age', 'Weight_kg', 'Height_m', 'Frequency', "Type", 'Level_x']])
    
    # Predecir los ejercicios
    predicciones = model.predict_proba(user_data)
    top_indices = np.argsort(predicciones, axis=1)[:, -num_ejercicios*dias:][0]
    
    # Obtener los títulos de los ejercicios recomendados
    ejercicios_recomendados = y.iloc[top_indices].values
    
    # Asegurar que no se repitan ejercicios el mismo día ni al día siguiente
    rutina_semanal = []
    usados = set()
    for i in range(dias):
        dia_ejercicios = []
        for ejercicio in ejercicios_recomendados:
            if ejercicio not in usados and len(dia_ejercicios) < num_ejercicios:
                dia_ejercicios.append(ejercicio)
                usados.add(ejercicio)
        rutina_semanal.append(dia_ejercicios)
    
    return rutina_semanal

KeyboardInterrupt: 